In [1]:
using Pkg
pkg_folder = "../"
Pkg.activate(pkg_folder)
using BenchmarkTools
import Markdown 
Base.showable(::MIME"text/markdown", ::Markdown.MD) =false # same function as below
include(pkg_folder*"/bnc_initialize.jl")

  Activating project at `~/Realizibility_index/bnc_julia`


show_qK_space_constrains (generic function with 1 method)

From the formal analysis, we find the one role the "non-asymptotic" vertex plays is that it builds a bridge for us to link two different "singular" veretx 

In [2]:
#    D₁ R₃ T₁ CD₁ CT₁
N = [1  2  0   -1  0
     0  2  1   0  -1]
model = Bnc(N=N)
[model.L
model.N]
# Dt, Rt, Tt, K1, K2

5×5 Matrix{Int64}:
 1  0  0   1   0
 0  1  0   2   2
 0  0  1   0   1
 1  2  0  -1   0
 0  2  1   0  -1

We especially care about while it will form: 
$$\begin{bmatrix}
0&0&0&1\\
0&0&1&0
\end{bmatrix}$$
$$\begin{bmatrix}
0&0&y&x\\
0&0&z&0
\end{bmatrix}$$

In [3]:
x = randomize(5) .|> log10

5-element Vector{Float64}:
  5.836596788817815
 -5.762617168193433
  0.47430111023800414
  3.7947271397590523
  2.95177125146097

In [4]:
logK1 = 0
logK2 = logK1-eps() # to avoid singularity

logTt_rg = (-5,5) #Titration 
logGt_rg = (-5,5) # Gene
logRt_rg = (-5,5) # Repressior

total_points = 5000000
point_num_eachdim = round(Int, total_points^(1/3))
mid_point = point_num_eachdim ÷ 2 + 1

logTt_step = range(logTt_rg...,length=point_num_eachdim)
logGt_step = range(logGt_rg...,length=point_num_eachdim)
logRt_step = range(logRt_rg...,length=point_num_eachdim)


-5.0:0.058823529411764705:5.0

In [5]:
qK2x(model,[-1.3051518200618322, -5.0, -5.0, 0.0, -2.220446049250313e-16],input_logspace=true,output_logspace=true)

5-element Vector{Float64}:
  -1.3051518201052617
  -5.000000430278375
  -5.000000000043429
 -11.305152680662012
 -15.000000860600178

In [6]:
#idx1: logE_min ~logE_max
#idx2: logS_min ~logS_max
#idx3: logK2_min ~logK2_max

#Initialize the array
logx_mt = Array{Vector{Float64},3}(undef, point_num_eachdim, point_num_eachdim, point_num_eachdim)

#Fill the first axis 
start_logqK = [logGt_rg[1], logRt_rg[1], logTt_rg[1], logK1 , logK2]
end_logqK = [logGt_rg[2], logRt_rg[1], logTt_rg[1], logK1 , logK2]
#The first axis is assigned with logGt's change.
logx_mt[:,1,1] .= x_traj_with_qK_change(model,start_logqK, end_logqK;input_logspace=true, output_logspace=true,saveat = range(0,1,length=point_num_eachdim),ensure_manifold=true)[2]

#Fill the first surface

Threads.@threads for i in 1:point_num_eachdim
    start_logqK = [logGt_step[i], logRt_rg[1], logTt_rg[1], logK1 , logK2]
    end_logqK = [logGt_step[i], logRt_rg[2], logTt_rg[1], logK1 , logK2]
    logx_mt[i,:,1] .= x_traj_with_qK_change(model, start_logqK, end_logqK; startlogx = logx_mt[i,1,1],input_logspace=true, output_logspace=true ,saveat = range(0,1,length=point_num_eachdim),ensure_manifold=true)[2]
end
# the second axis is assigned with logRt's change.


#Fill the whole cube
Threads.@threads for i in 1:point_num_eachdim # logGt
    for j in 1:point_num_eachdim # logRt
        start_logqK = [logGt_step[i], logRt_step[j], logTt_rg[1], logK1 , logK2]
        end_logqK = [logGt_step[i], logRt_step[j], logTt_rg[2], logK1 , logK2]
        logx_mt[i,j,:] .= x_traj_with_qK_change(model, start_logqK, end_logqK; startlogx = logx_mt[i,j,1],input_logspace=true, output_logspace=true ,saveat = range(0,1,length=point_num_eachdim),ensure_manifold=true)[2]
        # logx_mt[i,j,:] .= x_traj_with_qK_change(model, start_logqK, end_logqK;input_logspace=true, output_logspace=true ,saveat = range(0,1,length=point_num_eachdim))[2]
    end
end
# the third axis is assigned with logTt's change.

In [8]:
logqK_mt = Array{Vector{Float64},3}(undef, point_num_eachdim, point_num_eachdim, point_num_eachdim)
Threads.@threads for i in 1:point_num_eachdim # logGt
    for j in 1:point_num_eachdim # logRt
        for k in 1:point_num_eachdim # logTt
            logqK_mt[i,j,k] = [logGt_step[i], logRt_step[j], logTt_step[k], logK1 , logK2]
        end
    end
end

In [9]:
qK_rgm_asym = logqK_mt .|> x-> assign_vertex_qK(model;qK=x,input_logspace=true,asymptotic=true) |> x->get_idx(model,x)
qK_rgm = logqK_mt .|> x-> assign_vertex_qK(model;qK=x,input_logspace=true,asymptotic=false ) |> x->get_idx(model,x)

x_rgm_asym = logx_mt .|> x-> assign_vertex_x(model,x,input_logspace=true,asymptotic=true) |> x->get_idx(model,x)
x_rgm = logx_mt .|> x-> assign_vertex_x(model,x,input_logspace=true,asymptotic=false) |> x->get_idx(model,x)

(minval,maxval) = values(model.vertices_idx) |> x->(minimum(x),maximum(x))

Start finding all vertices, it may takes a while.
Done, with 12 vertices found and 12 real vertices.


(1, 12)

In [10]:
idx = 3
perm = get_perm(model,idx)
is_real = get_vertex!(model,idx).real
nullity = get_nullity!(model,idx)
println("idx=$idx,perm=$perm, is_real=$is_real, nullity=$nullity")

Start calculating vertex neighbor matrix, It may takes a while.
Done.
idx=3,perm=Int8[4, 2, 3], is_real=true, nullity=0


In [31]:
using ImageFiltering
function find_bounds(lattice)
    col_asym_x_bounds = imfilter(lattice, Kernel.Laplacian(), "replicate") # findboundary
    edge_map = col_asym_x_bounds .!= 0
    return edge_map
end

find_bounds (generic function with 1 method)

In [32]:
using GLMakie

# 参数
nslices = 11   # 可以随便改 slice 数目
slice_positions = round.(Int, range(1, point_num_eachdim, length=nslices+2)[2:end-1])

f = Figure(size=(600*nslices, 2000))

cmap = :rainbow
nlevels = maxval - minval + 1
cmap_disc = cgrad(cmap, nlevels, categorical=true)
crange = (minval, maxval)

# --- 3D 图 ---
p11 = Axis3(f[1, 1])
p11.title = "x space non-asym"
volume!(p11, x_rgm; colormap=cmap, colorrange=crange)

p12 = Axis3(f[2, 1])
p12.title = "qK space non-asym"
volume!(p12, qK_rgm; colormap=cmap, colorrange=crange)

p13 = Axis3(f[3, 1])
p13.title = "x space asym"
volume!(p13, x_rgm_asym; colormap=cmap, colorrange=crange)

p14 = Axis3(f[4, 1])
p14.title = "qK space asym"
volume!(p14, qK_rgm_asym; colormap=cmap, colorrange=crange)


logTt_step = range(logTt_rg..., length=point_num_eachdim)
# --- slices 横向排列 ---
# for (i, idx) in enumerate(slice_positions)
#     # x space slice
#     ax = Axis(f[1, i+1])
#     ax.title = "x slice $i,with logTt = $(round(logTt_step[idx],digits=2))"
#     heatmap!(ax, logGt_step, logRt_step, x_rgm[:, :, idx]; colormap=cmap, colorrange=crange)

#     # qK space slice
#     ax2 = Axis(f[2, i+1])
#     ax2.title = "qK slice $i,with logTt = $(round(logTt_step[idx],digits=2))"
#     heatmap!(ax2, logGt_step, logRt_step, qK_rgm[:, :, idx]; colormap=cmap, colorrange=crange)

#     ax3 = Axis(f[3, i+1])
#     ax3.title = "x slice asym $i,with logTt = $(round(logTt_step[idx],digits=2))"
#     heatmap!(ax3, logGt_step, logRt_step, x_rgm_asym[:, :, idx]; colormap=cmap, colorrange=crange)

#     ax4 = Axis(f[4, i+1])
#     ax4.title = "qK slice asym $i,with logTt = $(round(logTt_step[idx],digits=2))"
#     bounds = find_bounds(qK_rgm_asym[:, :, idx])

#     # heatmap!(ax4, logGt_step, logRt_step, qK_rgm_asym[:, :, idx]; colormap=cmap, colorrange=crange)
#     heatmap!(ax4, logGt_step, logRt_step, (logx_mt[:, :, idx] .|> x->x[1]))
#     # contour!(ax4, logGt_step, logRt_step,bounds,color = :black)
# end

for (i, idx) in enumerate(slice_positions)
    # x space slice
    ax = Axis(f[1, i+1])
    ax.title = "x slice $i,with logGt = $(round(logGt_step[idx],digits=2))"
    heatmap!(ax, logRt_step, logTt_step, x_rgm[idx, :, :]; colormap=cmap, colorrange=crange)

    # qK space slice
    ax2 = Axis(f[2, i+1])
    ax2.title = "qK slice $i,with logGt = $(round(logGt_step[idx],digits=2))"
    heatmap!(ax2, logRt_step, logTt_step, qK_rgm[idx, :, :]; colormap=cmap, colorrange=crange)

    ax3 = Axis(f[3, i+1])
    ax3.title = "x slice asym $i,with logGt = $(round(logGt_step[idx],digits=2))"
    heatmap!(ax3, logRt_step, logTt_step, x_rgm_asym[idx, :, :]; colormap=cmap, colorrange=crange)

    ax4 = Axis(f[4, i+1])
    ax4.title = "qK slice asym $i,with logGt = $(round(logGt_step[idx],digits=2))"
    bounds = find_bounds(qK_rgm_asym[idx, :, :])

    # heatmap!(ax4, logGt_step, logRt_step, qK_rgm_asym[:, :, idx]; colormap=cmap, colorrange=crange)
    heatmap!(ax4, logRt_step, logTt_step, (logx_mt[idx, :, :] .|> x->(x[1]-logGt_step[idx]) ))
    # contour!(ax4, logGt_step, logRt_step,bounds,color = :black)
    for ax in [ax, ax2, ax3, ax4]
        ax.xlabel = "logRt/logK"
        ax.ylabel = "logTt/logK"
    end
end


# label
# for p in [p11, p12]
#     p.xlabel = "logGt/logK"
#     p.ylabel = "logRt/logK"
#     p.zlabel = "logTt/logK"
# end

for p in [p11, p12, p13]
    p.xlabel = "logGt/logK"
    p.ylabel = "logRt/logK"
    p.zlabel = "logTt/logK"
end

# for ax in content(f)
#     if ax isa Axis
#         ax.xlabel = "logE"
#         ax.ylabel = "logS"
#     end
# end

# colorbar
Colorbar(f[:, end+1], colorrange=crange, colormap=cmap_disc, ticks=minval:maxval)
# Colorbar(f[:, end+1],colorrange = )
# save("/home/joker/rgm_slices_and_volume_of_Repressilator.png", f)
display(f)


GLMakie.Screen(...)

In [11]:
using GLMakie

# 参数
nslices = 11   # 可以随便改 slice 数目
slice_positions = round.(Int, range(1, point_num_eachdim, length=nslices+2)[2:end-1])

f = Figure(size=(600*nslices, 2000))

cmap = :rainbow
nlevels = maxval - minval + 1
cmap_disc = cgrad(cmap, nlevels, categorical=true)
crange = (minval, maxval)

# --- 3D 图 ---
p11 = Axis3(f[1, 1])
p11.title = "x space non-asym"
volume!(p11, x_rgm; colormap=cmap, colorrange=crange)

p12 = Axis3(f[2, 1])
p12.title = "x space asym"
volume!(p12, x_rgm_asym; colormap=cmap, colorrange=crange)

p13 = Axis3(f[3, 1])
p13.title = "qK space non-asym"
volume!(p13, qK_rgm; colormap=cmap, colorrange=crange)

p14 = Axis3(f[4, 1])
p14.title = "qK space asym"
volume!(p14, qK_rgm_asym; colormap=cmap, colorrange=crange)

for p in [ p11, p12, p13, p14]
    p.xlabel = "logGt-logK"
    p.ylabel = "logRt-logK"
    p.zlabel = "logTt-logK"
end


# --- slices 横向排列 ---
for (i, idx) in enumerate(slice_positions)
    # x space slice
    ax = Axis(f[1, i+1])
    ax.title = "x slice $i,with logTt = $(round(logTt_step[idx],digits=2))"
    heatmap!(ax, logGt_step, logRt_step, x_rgm[:, :, idx]; colormap=cmap, colorrange=crange)

    ax2 = Axis(f[2, i+1])
    ax2.title = "x slice asym $i,with logTt = $(round(logTt_step[idx],digits=2))"
    heatmap!(ax2, logGt_step, logRt_step, x_rgm_asym[:, :, idx]; colormap=cmap, colorrange=crange)
    
    
    # qK space slice
    ax3 = Axis(f[3, i+1])
    ax3.title = "qK slice $i,with logTt = $(round(logTt_step[idx],digits=2))"
    heatmap!(ax3, logGt_step, logRt_step, qK_rgm[:, :, idx]; colormap=cmap, colorrange=crange)
    # @show qK_rgm[:, :, idx][1,end]

    #qK space slice2
    ax4 = Axis(f[4, i+1])
    ax4.title = "qK slice asym $i,with logTt = $(round(logTt_step[idx],digits=2))"
    # heatmap!(ax4, logGt_step, logRt_step, qK_rgm_asym[:, :, idx]; colormap=cmap, colorrange=crange)
    # @show qK_rgm_asym[:, :, idx][1,end]

    # bounds = find_bounds(qK_rgm_asym[:, :, idx])

    function fff(x)
        g = copy(x)
        for col in eachcol(g)
            col .-= logGt_step
        end
        return g
    end
    
    # 
    heatmap!(ax4, logGt_step, logRt_step, (logx_mt[:, :, idx] .|> x->x[1]) |> fff )
    # contour!(ax4, logGt_step, logRt_step,bounds,color = :black)
    # for ax in [ax, ax2, ax3, ax4]
    for ax in [ax2, ax4]
        ax.xlabel = "logGt-logK"
        ax.ylabel = "logRt-logK"
    end
end

# for (i, idx) in enumerate(slice_positions)
#     # x space slice
#     ax = Axis(f[1, i+1])
#     ax.title = "x slice $i,with logGt = $(round(logGt_step[idx],digits=2))"
#     heatmap!(ax, logRt_step, logTt_step, x_rgm[idx, :, :]; colormap=cmap, colorrange=crange)

#     # qK space slice
#     ax2 = Axis(f[2, i+1])
#     ax2.title = "qK slice $i,with logGt = $(round(logGt_step[idx],digits=2))"
#     heatmap!(ax2, logRt_step, logTt_step, qK_rgm[idx, :, :]; colormap=cmap, colorrange=crange)

#     ax3 = Axis(f[3, i+1])
#     ax3.title = "x slice asym $i,with logGt = $(round(logGt_step[idx],digits=2))"
#     heatmap!(ax3, logRt_step, logTt_step, x_rgm_asym[idx, :, :]; colormap=cmap, colorrange=crange)

#     ax4 = Axis(f[4, i+1])
#     ax4.title = "qK slice asym $i,with logGt = $(round(logGt_step[idx],digits=2))"
#     bounds = find_bounds(qK_rgm_asym[idx, :, :])

#     # heatmap!(ax4, logGt_step, logRt_step, qK_rgm_asym[:, :, idx]; colormap=cmap, colorrange=crange)
#     heatmap!(ax4, logRt_step, logTt_step, (logx_mt[idx, :, :] .|> x->(x[1]-logGt_step[idx]) ))
#     # contour!(ax4, logGt_step, logRt_step,bounds,color = :black)
#     for ax in [ax, ax2, ax3, ax4]
#         ax.xlabel = "logRt/logK"
#         ax.ylabel = "logTt/logK"
#     end
# end


# colorbar
Colorbar(f[:, end+1], colorrange=crange, colormap=cmap_disc, ticks=minval:maxval)
# Colorbar(f[:, end+1],colorrange = )
# save("/home/joker/rgm_slices_and_volume_of_Repressilator_for_Jiarui.png", f)
display(f)


GLMakie.Screen(...)

In [21]:
for index in  1:length(find_all_vertices!(model))
    vtx = get_vertex!(model,index)
    perm = vtx.perm
    nullity = vtx.nullity
    real = vtx.real
    # println("Vertex $index: perm=$perm, singularity=$singularity, real=$real")
    println("Vertex $index: perm=$perm, nullity=$nullity")
end

Start calculating vertex neighbor matrix, It may takes a while.
Done.
Vertex 1: perm=Int8[1, 2, 3], nullity=0
Vertex 2: perm=Int8[1, 2, 5], nullity=0
Vertex 3: perm=Int8[4, 2, 3], nullity=0
Vertex 4: perm=Int8[4, 2, 5], nullity=0
Vertex 5: perm=Int8[1, 4, 3], nullity=0
Vertex 6: perm=Int8[1, 4, 5], nullity=0
Vertex 7: perm=Int8[4, 4, 3], nullity=1
Vertex 8: perm=Int8[4, 4, 5], nullity=1
Vertex 9: perm=Int8[1, 5, 3], nullity=0
Vertex 10: perm=Int8[1, 5, 5], nullity=1
Vertex 11: perm=Int8[4, 5, 3], nullity=0
Vertex 12: perm=Int8[4, 5, 5], nullity=1


In [22]:
get_C_C0_qK!(model,1)[1]

4×5 SparseMatrixCSC{Float64, Int64} with 10 stored entries:
   ⋅   -2.0    ⋅   1.0   ⋅ 
 -1.0  -1.0    ⋅   1.0   ⋅ 
   ⋅   -1.0  -1.0   ⋅   1.0
   ⋅   -2.0    ⋅    ⋅   1.0

In [27]:
get_C_C0_qK!(model,9)[1]

4×5 SparseMatrixCSC{Float64, Int64} with 12 stored entries:
   ⋅     ⋅   1.0   ⋅   -1.0
   ⋅   -1.0  1.0  1.0  -1.0
 -1.0    ⋅   1.0  1.0  -1.0
 -1.0    ⋅   1.0   ⋅     ⋅ 

In [28]:
get_M_M0!(model,1)[1]

5×5 SparseMatrixCSC{Int64, Int64} with 9 stored entries:
 1  ⋅  ⋅   ⋅   ⋅
 ⋅  1  ⋅   ⋅   ⋅
 ⋅  ⋅  1   ⋅   ⋅
 1  1  ⋅  -1   ⋅
 1  ⋅  1   ⋅  -1

In [29]:
get_M_M0!(model,9)[1]

5×5 SparseMatrixCSC{Int64, Int64} with 9 stored entries:
 ⋅  ⋅  ⋅   ⋅   1
 ⋅  1  ⋅   ⋅   ⋅
 ⋅  ⋅  1   ⋅   ⋅
 1  1  ⋅  -1   ⋅
 1  ⋅  1   ⋅  -1

In [36]:
get_H!(model,1)

5×5 SparseMatrixCSC{Float64, Int64} with 9 stored entries:
 1.0   ⋅    ⋅     ⋅     ⋅ 
  ⋅   1.0   ⋅     ⋅     ⋅ 
  ⋅    ⋅   1.0    ⋅     ⋅ 
 1.0  1.0   ⋅   -1.0    ⋅ 
 1.0   ⋅   1.0    ⋅   -1.0

In [37]:
get_H!(model,9)

5×5 SparseMatrixCSC{Float64, Int64} with 11 stored entries:
 1.0   ⋅   -1.0    ⋅   1.0
  ⋅   1.0    ⋅     ⋅    ⋅ 
  ⋅    ⋅    1.0    ⋅    ⋅ 
 1.0  1.0  -1.0  -1.0  1.0
 1.0   ⋅     ⋅     ⋅    ⋅ 

In [43]:
get_M_M0!(model,8)[1]

5×5 SparseMatrixCSC{Int64, Int64} with 9 stored entries:
 ⋅  ⋅  ⋅   1   ⋅
 ⋅  ⋅  ⋅   1   ⋅
 ⋅  ⋅  ⋅   ⋅   1
 1  1  ⋅  -1   ⋅
 1  ⋅  1   ⋅  -1

In [59]:
get_M_M0!(model,12)[1]

5×5 SparseMatrixCSC{Int64, Int64} with 9 stored entries:
 ⋅  ⋅  ⋅   1   ⋅
 ⋅  ⋅  ⋅   ⋅   1
 ⋅  ⋅  ⋅   ⋅   1
 1  1  ⋅  -1   ⋅
 ⋅  1  1   ⋅  -1

In [30]:
# using LinearAlgebra, JuMP, HiGHS

"""
    find_interface_normal(C1, C01, C2, C02; eps=1e-8)

自动寻找两个 regime 的交界超平面，并返回：
- n0: 单位法向量（指向 regime1 的一侧）
- c0: 平面距离原点的shift
- active_rows: 哪些约束是紧约束

要求：交界的法向空间必须是一维，否则报错。
"""
function find_interface_normal(C1, C2; eps=1e-8)
    m1, n = size(C1)
    m2, _ = size(C2)

    function form_superplane(c1, c2; tol=1e-9)
        k = nothing
        for a in 1:length(c1)
            i, j = c1[a], c2[a]
            if (i == 0 && j != 0) || (i != 0 && j == 0)
                return false
            elseif i == 0 && j == 0
                continue
            end
            k_new = i/j
            if k_new > 0
                return false
            elseif k === nothing
                k = k_new
            elseif abs(k_new - k) > tol
                return false
            end
        end
        return k !== nothing
    end

    # Step 1: 枚举所有候选行对
    found_c2 = nothing
    # found_c01 = nothing
    found_idx = nothing
    count = 0
    for i in 1:m1
        for j in 1:m2
            c1 = C1[i,:]#, C01[i]
            c2 = C2[j,:]#, C02[j]
            if form_superplane(c1, c2)
                count += 1
                @show Array(c1), Array(c2)
                if count > 1
                    error("找到了多个候选的 1D 公共法向量")
                end
                found_c2, found_idx = c2, (i,j)
            end
        end
    end
    
    if count == 0
        error("没有找到唯一的 1D 公共法向量")
    else
        return (found_c2, found_idx)
    end
end



find_interface_normal

In [28]:
get_C_C0_qK!(model,1)[1]

4×5 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
   ⋅   -1.0    ⋅   1.0   ⋅ 
 -1.0    ⋅     ⋅   1.0   ⋅ 
   ⋅     ⋅   -1.0   ⋅   1.0
   ⋅   -1.0    ⋅    ⋅   1.0

In [30]:
get_C_C0_qK!(model,4)[1]

4×5 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
   ⋅   1.0    ⋅   -1.0    ⋅ 
 -1.0  1.0    ⋅     ⋅     ⋅ 
   ⋅   1.0  -1.0    ⋅     ⋅ 
   ⋅   1.0    ⋅     ⋅   -1.0

In [34]:
function find_direction_direct(Bnc::Bnc{T},from,to) where T
    (C1,_) = get_C_C0_qK!(model,from)
    (C2,_) = get_C_C0_qK!(model,to)
    find_interface_normal(C1,C2)
end

find_direction_direct (generic function with 1 method)

In [35]:
find_direction_direct(model,1,9)

(Array(c1), Array(c2)) = ([0.0, 0.0, -1.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0, -1.0])


(sparsevec([3, 5], [1.0, -1.0], 5), (2, 1))